In [102]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import csv

In [103]:
m = gp.Model('Maritime Inventory Routing Problem')

# Creating classes in order organize the code

In [116]:
'''Simplifying by having inf max inventory for all ports.'''
class Port:
    def __init__(self, number, berths, isLoadingPort):
        self.number = number
        self.berths = berths
        self.max_inventory = np.inf
        self.loadingPort = isLoadingPort
        
    def __repr__(self):
        return f'Port {self.number}'


class Node:
    def __init__(self, port, time):
        self.port = port
        self.time = time
        if port==None:
            self.tuple = (None, time)
        else:
            self.tuple = (port.number, time)
        self.incoming_arcs = set()
        self.outgoing_arcs = set()
        self.berths = port.berths
    
    def __repr__(self):
        return str(self.tuple)


class Arc:
    '''Lack distance'''
    '''Cost should be fixed to the arc for basic MIRP.
    In MIRPSO, the cost is a function of speed and distance.'''
    def __init__(self, origin_node, destination_node):
        self.origin_node = origin_node
        self.destination_node = destination_node
        self.tuple = (origin_node, destination_node)
        # self.cost = cost
    
    def __repr__(self):
        return str(self.origin_node) + ' -> ' + str(self.destination_node)

class Vessel:
    def __init__(self, max_inventory, initial_inventory, max_operating_quantity):
        self.max_inventory = max_inventory
        self.inventory = initial_inventory
        self.max_operating_quantity = max_operating_quantity
        

## Read in metadata

In [115]:
# Function to parse the metadata from the file content
def parse_metadata(content):
    metadata = {}
    
    # Extract lines between "----- MetaData -----" and the next section separator (empty line)
    metadata_section = content[content.index("----- MetaData -----\n")+1 : content.index("\n", content.index("----- MetaData -----\n"))]
    
    for line in metadata_section:
        if ":" in line:
            # Split the line at ":" to extract the key and value
            key, value = line.split(":")
            # Store the key-value pair in the dictionary
            metadata[key.strip()] = value.strip()
    
    return metadata


In [106]:
def read_and_assign_metadata(file_path):
    # Read the file content
    with open(file_path, 'r') as file:
        content = file.readlines()

    # Parse the metadata
    metadata = parse_metadata(content)
    
    # Assign the metadata values to variables
    numPeriods = int(metadata['numPeriods'])
    numCommodities = int(metadata['numCommodities'])
    numLoadingRegions = int(metadata['numLoadingRegions'])
    numDischargingRegions = int(metadata['numDischargingRegions'])
    numLoadingPortsInRegion = [int(x) for x in metadata['numLoadingPortsInRegion'][1:-1].split()]
    numDischargingPortsInRegion = [int(x) for x in metadata['numDischargingPortsInRegion'][1:-1].split()]
    numVesselClasses = int(metadata['numVesselClasses'])
    numTermVesselsInClass = [int(x) for x in metadata['numTermVesselsInClass'][1:-1].split()]
    hoursPerPeriod = int(metadata['hoursPerPeriod'])
    spotMarketPricePerUnit = float(metadata['spotMarketPricePerUnit'])
    spotMarketDiscountFactor = float(metadata['spotMarketDiscountFactor'])
    perPeriodRewardForFinishingEarly = float(metadata['perPeriodRewardForFinishingEarly'])
    attemptCost = float(metadata['attemptCost'])
    constantForSinglePeriodAlphaSlack = float(metadata['constantForSinglePeriodAlphaSlack'])
    constantForCumulativeAlphaSlack = float(metadata['constantForCumulativeAlphaSlack'])
    
    # Return the assigned variables
    return {
        'numPeriods': numPeriods,
        'numCommodities': numCommodities,
        'numLoadingRegions': numLoadingRegions,
        'numDischargingRegions': numDischargingRegions,
        'numLoadingPortsInRegion': numLoadingPortsInRegion,
        'numDischargingPortsInRegion': numDischargingPortsInRegion,
        'numVesselClasses': numVesselClasses,
        'numTermVesselsInClass': numTermVesselsInClass,
        'hoursPerPeriod': hoursPerPeriod,
        'spotMarketPricePerUnit': spotMarketPricePerUnit,
        'spotMarketDiscountFactor': spotMarketDiscountFactor,
        'perPeriodRewardForFinishingEarly': perPeriodRewardForFinishingEarly,
        'attemptCost': attemptCost,
        'constantForSinglePeriodAlphaSlack': constantForSinglePeriodAlphaSlack,
        'constantForCumulativeAlphaSlack': constantForCumulativeAlphaSlack
    }

# Read the metadata from the dummy data file and assign to variables
variables = read_and_assign_metadata('data/dummy_data.txt')

variables


{'numPeriods': 3,
 'numCommodities': 1,
 'numLoadingRegions': 1,
 'numDischargingRegions': 1,
 'numLoadingPortsInRegion': [3],
 'numDischargingPortsInRegion': [3],
 'numVesselClasses': 1,
 'numTermVesselsInClass': [3],
 'hoursPerPeriod': 24,
 'spotMarketPricePerUnit': 1.0,
 'spotMarketDiscountFactor': 0.999,
 'perPeriodRewardForFinishingEarly': 0.01,
 'attemptCost': 0.01,
 'constantForSinglePeriodAlphaSlack': 0.5,
 'constantForCumulativeAlphaSlack': 1.0}

Metadata loaded in. Use only the most important data in the beginning. 

- numPeriods
- numTermVesselsInClass (If only 1 numVesselClasses we have homogenous fleet.)
- Set of ports:
    - numLoadingRegions x numLoadingPortsInRegion
    - numDischargingPortsInRegion x numDischargingPortsInRegion
- Set of regular nodes
    - Needs to be created based on the timeperiods and ports.
- Set of all nodes
    - Add sink and source node to the set of regular nodes.

- The arcs do also need to be included

# Initial Parameters

All parameters should be set below

In [107]:
# Time periods
numTimePeriods = variables['numPeriods']

# Number of vessels
numVessels = variables['numTermVesselsInClass'][0]

# Number of ports
numLoadingPorts = 0
for i in range(variables['numLoadingRegions']):
    numLoadingPorts += variables['numLoadingPortsInRegion'][i]

numDischargingPorts = 0
for i in range(variables['numDischargingRegions']):
    numDischargingPorts += variables['numDischargingPortsInRegion'][i]


In [108]:
# Create ports. Only 1 berth per port for now.
ports = []
for i in range(1,numLoadingPorts+1):
    port = Port(number=i, berths=1, isLoadingPort=True)
    ports.append(port)

for i in range(1,numDischargingPorts+1):
    port = Port(number=i+numLoadingPorts, berths=1, isLoadingPort=False)
    ports.append(port)
    
for port in ports:
    print(port)

Port 1
Port 2
Port 3
Port 4
Port 5
Port 6


In [109]:
# Create the vessels. Fixed max inventory and max_operating_quantity for now.
vessels = []
for i in range(numVessels):
    vessel = Vessel(max_inventory=300, initial_inventory=0, max_operating_quantity=300)
    vessels.append(vessel)

In [110]:
# Create the regular nodes
regularNodes = []
for t in range(1, numTimePeriods+1):
    for port in ports:
        node = Node(port=port, time=t)
        regularNodes.append(node)
    
    
# Create fictional source and sink port
sourcePort = Port(number=0, berths=len(vessels), isLoadingPort=True)
sinkPort = Port(number=len(ports)+1, berths=len(vessels), isLoadingPort=False)

# Create source and sink node
sourceNode = Node(port=sourcePort, time=0)
sinkNode = Node(port=sinkPort, time=numTimePeriods+1)

nodes = [sourceNode] + regularNodes + [sinkNode]

# Sets

Initializing sets

In [111]:
'''Using the parsed data instead'''

# Create a set of all incoming arcs to node n associated with vessel v in a time-space model
# There can only be an arc from a node to another node if the time period of the first node is less than the time period of the second node
def incoming_arcs(node, nodes):
    return {Arc(origin_node=i, destination_node=node) for i in nodes if i.time < node.time}

# Create a set of all outgoing arcs from node n associated with vessel v in a time-space model
# There can only be an arc from a node to another node if the time period of the first node is less than the time period of the second node
def outgoing_arcs(node, nodes):
    return {Arc(origin_node=node, destination_node=j) for j in nodes if node.time < j.time}

# Create the set of arcs associated with vessel v in a time-space model
# Note: The arcs need to be revised so that only feasible travels are included
def generate_all_arcs(nodes):
    arcs = set()
    
    # Arcs between all nodes
    for n in nodes:
        arcs.update(outgoing_arcs(n, nodes))
    return arcs

In [114]:
# Create the arcs
all_arcs = generate_all_arcs(nodes)

# Create the arcs for each node
for n in nodes:
    for a in all_arcs:
        if a.origin_node.tuple == n.tuple:
            n.outgoing_arcs.add(a)
        if a.destination_node.tuple == n.tuple:
            n.incoming_arcs.add(a)

All nodes and arcs have been created. However there are arcs between all nodes, even though it may be infeasible for a vessel to travel the distance in time. 
### Starting with Gurobi

In [113]:
'''Creating the variables'''
'''Binary first'''
# x is the binary variable that indicates whether a vessel travels on arc a, where and arc is a route frome one node to another node. 
x = m.addVars((arc.tuple for arc in all_arcs), vtype=GRB.BINARY, name="x")

# o is the binary variable that indicates whether vessel v is operating (loading/unloading) at port p at time t
o = m.addVars(((port.number, t, vessel) for port in ports for t in time_period_range for vessel in vessels), vtype=GRB.BINARY, name="o")

'''Continuous varibles'''
# q is the amount of product loaded or unloaded at port i by vessel v at time t
q_bounds = {(port.number, t, vessel): min(vessel.max_inventory, port.max_inventory) for port in ports for t in time_period_range for vessel in vessels}
q = m.addVars(q_bounds.keys(), lb=0, ub=q_bounds, vtype=GRB.CONTINUOUS, name="q")

# s is the amount of product at port i at the end of period t
s_bounds = {(port.number, t): port.max_inventory for port in ports for t in time_period_range}
s = m.addVars(s_bounds.keys(), lb=0, ub=s_bounds, vtype=GRB.CONTINUOUS, name="s")

# w is the amount of product on board of vessel v at the end of time period t
w_bounds = {(t, vessel): vessel.max_inventory for vessel in vessels for t in time_period_range}
w = m.addVars(w_bounds.keys(), lb=0, ub=w_bounds, vtype=GRB.CONTINUOUS, name="w")